# Фильмов и Кино Лингвистический корпус

Милена Камская, Мария Криволап, Анастасия Фирсова, Стефания Харская

2023

## Парсинг

Первый этап, которым мы займёмся: парсинг и сбор новостей. Создадим DF, куда будем по итогу записывать всю необходимую информацию. 

In [1]:
from fake_useragent import UserAgent
import requests
import random
from bs4 import BeautifulSoup
import re
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
ua = UserAgent(verify_ssl=False)

In [ ]:
session = requests.session()

In [ ]:
df_texts = pd.DataFrame({'title': [], 'author': [], 'pub_user': [], 'pub_tag': [], 'pub_day': [],
                         'pub_month': [], 'pub_year': [], 'pub_time': [], 'url': [], 'news': []})

Следующая функция отвечает за разбивку страницы. Здесь берётся название новости, дата, рубрика, пользователей, короткая часть новости (там вылезает \n её убираем.), сслыка на саму новость.

In [ ]:
def parse_news_page_block(one_block):
    block = {}
    a = one_block.find('a')
    info_string = one_block.find('div', {'class': 'infobar'}).text

    block['title'] = a.text

    block['href'] = a.attrs['href']

    block['pub_day'] = int(info_string[0:2])
    block['pub_month'] = int(info_string[3:5])
    block['pub_year'] = int(info_string[6:11])
    block['pub_time'] = str(info_string[11:16])

    block['pub_user'] = re.findall(r'\|\s([A-Za-z0-9\.\_\-]+)\s\|', info_string)

    tags = str(re.findall(r'\:\s([А-Яа-яЁё0-9\.\_\-\/\s]+)', info_string)[0])
    block['pub_tag'] = tags.split(' / ')
    return block

Здесь работаем с новостью. Возникла проблема, что автор не выделяется отдельно, а как бы написан в тексте новости. В самом начале достаём автора текста, а потом просто удаляем. Всё-таки нужна чистая новость.

In [ ]:
def parse_one_new(block):
    url_one = block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    full_text = soup.find('div', {'class': 'fullnewsclass'}).text
    full_text = full_text.replace('\n', '')

    block['auth'] = re.findall(r':\s([А-Яа-яёЁ]+\s[А-ЯЁ][а-яё]+)', full_text)

    block['full_text'] = str(re.sub(r'([А-ЯЁ][а-яё]+:\s[А-Яа-яёЁ]+\s[А-ЯЁ][а-яё]+)', '', full_text))
    block['full_text'] = block['full_text'].replace('Поделитесь с друзьями', '')


    return block

Здесь соединяем две предыдущие функции. Дётся ссылку на страницу, прогоняется по двум функциям выше. Если вылезает ошибка, то её записываем в файл и прикрепляем ссылку на новость (или на страницу с новостями, но такое вроде не встречается). Почему может вылезти ошибка?

Разное формирование текста новостей. Постарались учесть разные варианты в регулярке, но тем не менее в тексте не всегда указывается автор или формат может быть иным.

In [ ]:
def get_nth_page(page_number):

    url = f'https://allbestmovies.ru/novosti-kino/page/{page_number}/'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    news = soup.find_all('div', {'class': 'shortstory'})

    blocks = []
    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
            with open('Errors.txt', 'a', encoding='utf-8') as f:
                    error = ' '.join([str(e), 'Ссылка на страницу:', url, '\n'])
                    f.write(error)


    result = []
    for b in blocks:
        try:
            res = parse_one_new(b)
            result.append(res)
        except Exception as e:
            with open('Errors.txt', 'a', encoding='utf-8') as f:
                error = ' '.join([str(e), 'Ссылка на страницу:', url, '\n'])
                f.write(error)

    return result

Эта функция отвечает за то, что нам надо начать обрабатывать именно эту страницу с новостями + потом записываем в DF.

In [ ]:
def run_all(n_pages, df_texts):
    for i in tqdm(range(n_pages)):
        blocks = get_nth_page(i+1)
        for block in blocks:
            new = [block['title'], block['auth'], block['pub_user'],
                   block['pub_tag'], block['pub_day'], block['pub_month'],
                   block['pub_year'], block['pub_time'], block['href'], block['full_text']]
            df_texts = df_texts.append(pd.Series(new, index=df_texts.columns[:len(new)]), ignore_index=True)
    return df_texts

In [ ]:
df_texts = run_all(10, df_texts)

Запускаем функцию и обрабатываем несколько страниц новостей. В среднем на страницу 15 новостей, тексты на 100 слов, примерно => нужно около 10 страниц (с запасом)

А ещё неожиданным образом три колонки превратились в float, поэтому сделаем их всё-таки int.

In [ ]:
df_texts[["pub_day", "pub_month", "pub_year"]] = df_texts[["pub_day", "pub_month", "pub_year"]].astype(int)

In [ ]:
df_texts.to_csv(r'df_texts.csv', index= False )

## База данных

Второй этап. Организация базы данных.

In [13]:
import csv
import pandas as pd
import stanza
import sqlite3

In [14]:
stanza_pipeline = stanza.Pipeline("ru")

2023-10-25 14:18:44 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-25 14:18:48 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |
| ner       | wikiner            |

2023-10-25 14:18:48 INFO: Using device: cpu
2023-10-25 14:18:48 INFO: Loading: tokenize
2023-10-25 14:18:48 INFO: Loading: pos
2023-10-25 14:18:48 INFO: Loading: lemma
2023-10-25 14:18:49 INFO: Loading: depparse
2023-10-25 14:18:49 INFO: Loading: ner
2023-10-25 14:18:50 INFO: Done loading processors!


Создаем все таблицы в базе данных.

In [ ]:
con = sqlite3.connect("textbase2.db")
cur = con.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS texts
(text_id integer PRIMARY KEY AUTOINCREMENT, name text, url text, year int,
 author text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS sents
(sent_id integer PRIMARY KEY AUTOINCREMENT, sent text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS words
(word_id integer PRIMARY KEY AUTOINCREMENT, word text, lemma text, pos text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS texts_sents
(texts_sents_id INTEGER PRIMARY KEY AUTOINCREMENT, text_id int, sent_id int)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS sents_words
(sents_words_id INTEGER PRIMARY KEY AUTOINCREMENT, sent_id int, word_id int)
""")

Перебираем тексты новостей.

Для каждого текста i: добавляем в таблицу текстов метаинформацию, делим сам текст на предложения.

В тексте i для каждого предложения n: добавляем его в таблицу предложений, добавляем также в таблицу соответствий текстов-предложений пару n - i.  Анализируем предложение с помощью stanza, обращаемся к словам.

Для каждого слова w в предложении n: добавляем саму словоформу в таблицу слов, туда же записываем лемму и часть речи. И добавляем в таблицу соответствий предложений-слов пару n - w.

In [ ]:
def analyze_and_record(file):
    sent_counter = 0 # так будем следить за индексами
    word_counter = 0
    with open(file, encoding = 'utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for index, news in enumerate(reader):
            name = news["title"]
            text = news["news"]
            year = news["pub_year"]
            url = news["url"]
            text_id = index + 1
            if news["author"] != []: #проверяем наличие автора, если его нет, берем пользователя, написавшего новость (на сайте это разные люди)
                author = str(news["author"])
            else:
                author = str(news["pub_user"])
            cur.execute("INSERT INTO texts VALUES (?, ?, ?, ?, ?)", (text_id, name, url, year, author)) # добавляем метаинформацию
            con.commit()

            stanza_doc = stanza_pipeline(text)
            stanza_dict = {}
            for stanza_sent in stanza_doc.sentences:
                sent = stanza_sent
                sent_counter += 1
                cur.execute("INSERT INTO sents VALUES (?, ?)", (sent_counter, str(sent.text)))
                con.commit()
                cur.execute("INSERT INTO texts_sents VALUES (?, ?, ?)", (None, index + 1, sent_counter))
                con.commit()
                for word in sent.words:
                    if word.upos != "PUNCT": # нужны только словоформы
                        if word.text[-1] in '«./,"»': # фильтруем на всякий случай
                            word_text = word.text[:-1]
                        elif word.text[0] in '«./,"»':
                            word_text = word.text[1:]
                        else:
                            word_text = word.text
                        word_counter += 1
                        pos = word.upos
                        lemma = word.lemma
                        cur.execute("INSERT INTO words VALUES (?, ?, ?, ?)", (word_counter, str(word_text), str(lemma), str(pos)))
                        con.commit()
                        cur.execute("INSERT INTO sents_words VALUES (?, ?, ?)", (None, sent_counter, word_counter))
                        con.commit()


In [ ]:
analyze_and_record('df_texts_clean.csv')

## Функции поиска

Третий этап. Написание функций, которые будут отвечать за поиск.

In [3]:
con = sqlite3.connect('textbase2.db')
cur = con.cursor()

Функция find_quote ищет слово именно в такой форме, которую задал пользователь.

In [4]:
def find_quote(search_word):
    search_word = re.sub(r'"', '', search_word)
    search_words = []
    search_words.append(search_word.lower())
    search_words.append(search_word.lower().title())

    word_find = """
    SELECT *
    FROM words
        JOIN sents_words ON words.word_id = sents_words.word_id
        JOIN sents ON sents_words.sent_id = sents.sent_id
        JOIN texts_sents ON sents.sent_id = texts_sents.sent_id
        JOIN texts ON texts_sents.text_id = texts.text_id
        WHERE word = ?
    """
    all_sent = []
    for i in search_words:
        cur.execute(word_find, (i,))
        for element in cur.fetchall():
            all_sent.append(element)

    return all_sent

In [ ]:
find_quote('"Пока"')

Функция find_tag ищет тег, который задал пользователь. Тег должен быть задан латиницей и входить в UPOS.


In [6]:
def find_tag(search_tag):
    search_tag = search_tag.upper()
    sentences = []

    word_pos_find = """
    SELECT *
    FROM words
        JOIN sents_words ON words.word_id = sents_words.word_id
        JOIN sents ON sents_words.sent_id = sents.sent_id
        JOIN texts_sents ON sents.sent_id = texts_sents.sent_id
        JOIN texts ON texts_sents.text_id = texts.text_id
        WHERE pos = ?
    """
    cur.execute(word_pos_find, (search_tag,))
    all_sent = []
    for element in cur.fetchall():
        all_sent.append(element)

    return all_sent

In [ ]:
find_tag('nOun')

Функция find_word ищет все варианты слова, которое задал пользователь.

In [15]:
def find_word(search_word):
    search_word = search_word.lower()
    stanza_proc = stanza_pipeline(search_word)
    search_lemma = [word.lemma for sent in stanza_proc.sentences for word in sent.words]
    sentences = []

    word_lemma_find = """
    SELECT *
    FROM words
        JOIN sents_words ON words.word_id = sents_words.word_id
        JOIN sents ON sents_words.sent_id = sents.sent_id
        JOIN texts_sents ON sents.sent_id = texts_sents.sent_id
        JOIN texts ON texts_sents.text_id = texts.text_id
        WHERE lemma = ?
    """
    for lemma in search_lemma:
        cur.execute(word_lemma_find, (lemma,))

    all_sent = []
    for element in cur.fetchall():
        all_sent.append(element)

    return all_sent

In [ ]:
find_word('дом')

Функция find_quote_and_tag ищет лемму с определенным тегом, то для их разделения нужно использовать "+".

Если пользователь хочет найти слово именно в такой форме, которую задал пользователь, с определенным тегом, который тоже указывает пользователь. Они должны быть записанны друг за другом с помощью знака "=".

In [185]:
def find_quote_and_tag(search_word_and_tag):

    swt = re.sub('[=|+]',' ',search_word_and_tag)
    swt1 = re.sub(r'\s+', ' ', swt)

    if '=' in search_word_and_tag:
        words = []
        search = swt1.split(' ')

        words.append(search[0].lower())
        words.append(search[0].lower().title())
        tag = search[1].upper()

        word_and_tag_find = """
        SELECT *
        FROM words
            JOIN sents_words ON words.word_id = sents_words.word_id
            JOIN sents ON sents_words.sent_id = sents.sent_id
            JOIN texts_sents ON sents.sent_id = texts_sents.sent_id
            JOIN texts ON texts_sents.text_id = texts.text_id
            WHERE word = ? AND pos = ?
        """

        for i in words:
            cur.execute(word_and_tag_find, (i, tag))

        all_sent = []
        for element in cur.fetchall():
            all_sent.append(element)

    elif '+' in search_word_and_tag:
          search = swt1.split(' ')
          search_word = search[0].lower()
          stanza_proc = stanza_pipeline(search_word)
          search_lemma = [word.lemma for sent in stanza_proc.sentences for word in sent.words]

          tag = search[1].upper()

          word_and_tag_find = """
          SELECT *
          FROM words
              JOIN sents_words ON words.word_id = sents_words.word_id
              JOIN sents ON sents_words.sent_id = sents.sent_id
              JOIN texts_sents ON sents.sent_id = texts_sents.sent_id
              JOIN texts ON texts_sents.text_id = texts.text_id
              WHERE lemma = ? AND pos = ?
          """
          all_sent = []
          for lemma in search_lemma:
              cur.execute(word_and_tag_find, (lemma, tag))
              for element in cur.fetchall():
                  all_sent.append(element)

    return all_sent

In [ ]:
find_quote_and_tag('хотеть+verb')

In [19]:
def choose_func(part):
    result_tag = re.search(r'[A-Za-z]', part)

    if '+' in part or '=' in part:
        return find_quote_and_tag(part)
    elif result_tag:
        return find_tag(part)
    elif '"' in part:
        return find_quote(part)
    elif not result_tag:
        return find_word(part)
    else:
        return 'Неправильный формат ввода. Попробуйте ещё раз.'

In [179]:
def search(line_search):
    line = line_search.split()

    if len(line) == 1:
        dict_meta = {}
        elem = choose_func(line[0])
        for i in range(0, len(elem)):
            dict_meta[elem[i][8]] = [elem[i][1], elem[i][16], elem[i][13], elem[i][15], elem[i][14]]
        for sent,meta in dict_meta.items():
            print(f'{sent} [{meta[1]}, «{meta[2]}», {meta[3]}, {meta[4]}]\n')

    elif len(line) > 1:
        sentences_print = [] #предложения, которые будем выводить

        sw1 = {} # словари индекс предложения - индексы слов
        sw2 = {}
        sentences_1 = {} #множество предложений по первой части запроса (там точно будет нужное в итоге)
        dict_meta = {}
        sw_1_2 = {} # словарь индекс предложения для триграмм - индекс второго слова в триграмме

        first_word = choose_func(line[0]) #результат по первому слову
        second_word = choose_func(line[1])#результат по второму слову
        
        for i in range(0, len(first_word)): #заполнение списков
            sentences_1[first_word[i][5]] =  first_word[i][8]
            dict_meta[first_word[i][8]] = [first_word[i][1], first_word[i][16], first_word[i][13], first_word[i][15], first_word[i][14]]
            if first_word[i][5] not in sw1:
                sw1[first_word[i][5]] = []
                sw1[first_word[i][5]].append(first_word[i][0])
            else:
                sw1[first_word[i][5]].append(first_word[i][0])
        for i in range(0, len(second_word)):
            if second_word[i][5] not in sw2:
                sw2[second_word[i][5]] = []
                sw2[second_word[i][5]].append(second_word[i][0])
            else:
                sw2[second_word[i][5]].append(second_word[i][0])

        ind_fin = {} # словарь с индексами предложений, где есть результаты по каждой из частей запроса
        for ind in sw1:
            if ind in sw2:
                ind_fin[ind] = []
                ind_fin[ind].append(sw1[ind])
                ind_fin[ind].append(sw2[ind])

        for ind in ind_fin:
            for ind_1 in ind_fin[ind][0]:
                for ind_2 in ind_fin[ind][1]:
                    if ind_1 + 1 == ind_2:
                        sentences_print.append(sentences_1[ind])
                        sw_1_2[ind] = [ind_2]
                       
        if len(line) == 3:
            sw3 = {}
            third_word = choose_func(line[2]) #результат по третьему слову
            ind_fin_3 = {}
            sentences_print = []
            
            for i in range(0, len(third_word)):
                if third_word[i][5] not in sw3:
                    sw3[third_word[i][5]] = []
                    sw3[third_word[i][5]].append(third_word[i][0])
                else:
                    sw3[third_word[i][5]].append(third_word[i][0])
            
            for ind in sw_1_2:
                if ind in sw3:
                    ind_fin_3[ind] = []
                    ind_fin_3[ind].append(sw_1_2[ind])
                    ind_fin_3[ind].append(sw3[ind])

            for ind in ind_fin_3:
                for ind_2 in ind_fin_3[ind][0]:
                    for ind_3 in ind_fin_3[ind][1]:
                        if ind_2 + 1 == ind_3:
                            sentences_print.append(sentences_1[ind])

        for sent,meta in dict_meta.items():
            for item in sentences_print:
                if sent == item:
                    print(f'{sent} [{meta[1]}, «{meta[2]}», {meta[3]}, {meta[4]}]\n')
        

In [184]:
search('создатели verb')

Пока создатели довольствовались своим успехом, их сотрудники работали без выходных и за низкую зарплату. [Екатерина Павлова, «10 лучших трейлеров января», 2022, https://allbestmovies.ru/10768-10-luchshih-trejlerov-janvarja.html]

В роли Блэка создатели видят не белого актёра. [Виктория Гаржанова, «Лунатик, Бродяга, Сохатый и Хвост: новый американской сериал по вселенной Гарри Поттера ищет трансгендерных и небинарных актёров», 2022, https://allbestmovies.ru/10722-lunatik-brodjaga-sohatyj-i-hvost-novyj-amerikanskoj-serial-po-vselennoj-garri-pottera-ischet-transgendernyh-i-nebinarnyh-akterov.html]

Задача создателей познакомить аудиторию с классикой отечественного неигрового кино, а также сделать частью современного культурного процесса фильмы, на сегодняшний день доступные лишь узкому кругу специалистов и архивных работников. [София Маргацкая, «Неигровое кино ХХ века в современном прочтении представил портал Кино-Театр.Ру», 2021, https://allbestmovies.ru/10609-neigrovoe-kino-hh-veka-v-so